In [3]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from pathlib import Path
import csv
import warnings

# -------------------------
# Suppress NVML warning
# -------------------------
warnings.filterwarnings("ignore", message="Can't initialize NVML")

# -------------------------
# Paths & Model
# -------------------------
model = YOLO("/home/debasish/Documents/YOLOv8/weights/worms-and-eggs/yolov8l-832-eggs.pt")

input_path = "/home/debasish/Documents/YOLOv8/images/worms_and_eggs/test"  # image or video folder
output_base_dir = Path("/home/debasish/Documents/YOLOv8/outputs/counting")
output_base_dir.mkdir(parents=True, exist_ok=True)

VIDEO_EXTS = [".mp4", ".avi", ".mkv", ".mov"]
IMAGE_EXTS = [".png", ".jpg", ".jpeg", ".bmp", ".tif"]

# -------------------------
# Create run folder (run1, run2, ...) for each input
# -------------------------
def create_run_folder(base_dir):
    run_id = 1
    while (base_dir / f"run{run_id}").exists():
        run_id += 1
    run_dir = base_dir / f"run{run_id}"
    run_dir.mkdir(parents=True, exist_ok=True)
    return run_dir

# -------------------------
# Function to process image
# -------------------------
def process_image(image_path, run_dir):
    img_name = Path(image_path).stem
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"[ERROR] Cannot read image {image_path}")
        return

    results = model.predict(image, verbose=True, conf=0.3)

    worm_count, egg_count = 0, 0
    for result in results:
        boxes = result.boxes.xywh.cpu().numpy()
        classes = result.boxes.cls.cpu().numpy().astype(int)

        worm_count += np.sum(classes == 1)
        egg_count += np.sum(classes == 0)

        for box, cls in zip(boxes, classes):
            x_c, y_c, w, h = box
            x1 = int(x_c - w / 2)
            y1 = int(y_c - h / 2)
            x2 = int(x_c + w / 2)
            y2 = int(y_c + h / 2)
            color = (255, 0, 0) if cls == 1 else (0, 0, 255)
            label = "w" if cls == 1 else "e"
            cv2.rectangle(image, (x1, y1), (x2, y2), color, 1)
            cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    cv2.putText(image, f"Worms: {worm_count}", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(image, f"Eggs: {egg_count}", (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    save_path = run_dir / f"{img_name}_count.png"
    cv2.imwrite(str(save_path), image)

    # Save counts to CSV
    csv_path = run_dir / f"{img_name}_count.csv"
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["object", "count"])
        writer.writerow(["worm", worm_count])
        writer.writerow(["egg", egg_count])

    print(f"[INFO] Processed image: {img_name}, Worms: {worm_count}, Eggs: {egg_count}")
    print(f"[INFO] Saved image: {save_path}, CSV: {csv_path}")

# -------------------------
# Function to process video
# -------------------------
def process_video(video_path, run_dir):
    vid_name = Path(video_path).stem
    cap = cv2.VideoCapture(str(video_path))
    width, height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0 or fps > 120:
        fps = 30.0
    fps = round(fps, 2)

    # Try multiple codecs
    for codec in ["mp4v", "XVID", "avc1"]:
        fourcc = cv2.VideoWriter_fourcc(*codec)
        out = cv2.VideoWriter(str(run_dir / f"{vid_name}_count.mp4"), fourcc, fps, (width, height))
        if out.isOpened():
            print(f"[INFO] Using codec {codec}")
            break
    else:
        raise RuntimeError("Could not initialize VideoWriter with any supported codec.")

    frame_num = 0
    csv_path = run_dir / f"{vid_name}_count.csv"
    with open(csv_path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["frame", "worm_count", "egg_count"])

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_num += 1

            results = model.predict(frame, verbose=False, conf=0.3)
            worm_count, egg_count = 0, 0
            for result in results:
                boxes = result.boxes.xywh.cpu().numpy()
                classes = result.boxes.cls.cpu().numpy().astype(int)

                worm_count += np.sum(classes == 1)
                egg_count += np.sum(classes == 0)

                for box, cls in zip(boxes, classes):
                    x_c, y_c, w, h = box
                    x1 = int(x_c - w / 2)
                    y1 = int(y_c - h / 2)
                    x2 = int(x_c + w / 2)
                    y2 = int(y_c + h / 2)
                    color = (255, 0, 0) if cls == 1 else (0, 0, 255)
                    label = "w" if cls == 1 else "e"
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 1)
                    cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

            cv2.putText(frame, f"Worms: {worm_count}", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Eggs: {egg_count}", (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            out.write(frame)
            writer.writerow([frame_num, worm_count, egg_count])

    cap.release()
    out.release()
    print(f"[INFO] Processed video: {vid_name}")
    print(f"[INFO] Saved video: {run_dir / f'{vid_name}_count.mp4'}, CSV: {csv_path}")

# -------------------------
# Main runner
# -------------------------
input_path = Path(input_path)
if input_path.is_file():
    run_dir = create_run_folder(output_base_dir)
    if input_path.suffix.lower() in IMAGE_EXTS:
        process_image(input_path, run_dir)
    elif input_path.suffix.lower() in VIDEO_EXTS:
        process_video(input_path, run_dir)
elif input_path.is_dir():
    for f in input_path.iterdir():
        if f.suffix.lower() in IMAGE_EXTS + VIDEO_EXTS:
            run_dir = create_run_folder(output_base_dir)
            if f.suffix.lower() in IMAGE_EXTS:
                process_image(f, run_dir)
            else:
                process_video(f, run_dir)
else:
    print(f"[ERROR] No valid images or videos found at {input_path}")

[INFO] Using codec mp4v
[INFO] Processed video: image052
[INFO] Saved video: /home/debasish/Documents/YOLOv8/outputs/counting/run1/image052_count.mp4, CSV: /home/debasish/Documents/YOLOv8/outputs/counting/run1/image052_count.csv

0: 640x640 5 eggs, 5 worms, 13.7ms
Speed: 2.0ms preprocess, 13.7ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)
[INFO] Processed image: image303, Worms: 5, Eggs: 5
[INFO] Saved image: /home/debasish/Documents/YOLOv8/outputs/counting/run2/image303_count.png, CSV: /home/debasish/Documents/YOLOv8/outputs/counting/run2/image303_count.csv

0: 640x640 17 eggs, 5 worms, 11.9ms
Speed: 1.6ms preprocess, 11.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
[INFO] Processed image: frame, Worms: 5, Eggs: 17
[INFO] Saved image: /home/debasish/Documents/YOLOv8/outputs/counting/run3/frame_count.png, CSV: /home/debasish/Documents/YOLOv8/outputs/counting/run3/frame_count.csv
